In [1]:
# Import the requrired librarires
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time,os
#from tqdm import tqdm_notebook
from tqdm import tqdm

In [2]:
# Enter the file name to get the links from and the file name to save the scraped descriptions
# Modify this cell to download from various platforms
%cd ..
merged_df = pd.read_csv('datasets/platform_url_merged/switch_merged.csv')
save_file_name = 'switch_description'
# Go up a directory

C:\Users\Adnan\Desktop\Data Science\Week 13\NLP Project


In [3]:
# Have a peek at the source file
merged_df.head(10)

,title,url
0,#1 Anagrams (Switch),https://www.metacritic.com/game/switch/1-anagrams
1,#1 Crosswords (Switch),https://www.metacritic.com/game/switch/1-cross...
2,#1 Sudokus (Switch),https://www.metacritic.com/game/switch/1-sudokus
3,#Breakforcist Battle (Switch),https://www.metacritic.com/game/switch/breakfo...
4,#DRIVE (Switch),https://www.metacritic.com/game/switch/drive
5,#Funtime (Switch),https://www.metacritic.com/game/switch/funtime
6,"#Halloween, Super Puzzles Dream (Switch)",https://www.metacritic.com/game/switch/hallowe...
7,#KILLALLZOMBIES (Switch),https://www.metacritic.com/game/switch/killall...
8,"#NoLimitFantasy, Super Puzzles Dream (Switch)",https://www.metacritic.com/game/switch/nolimit...
9,"#pinocchio, Super Puzzles Dream (Switch)",https://www.metacritic.com/game/switch/pinocch...


In [4]:
len(merged_df)

9641

In [5]:
# Funciton to scrape the details from each link
def get_data(driver,title,url):
    try:
        driver.get(url)
    except:
        driver = web_driver()
        driver.get(url)
    try:
        name = driver.find_element(By.CSS_SELECTOR,'div.product_title > a:nth-child(1) > h1:nth-child(1)')
        title = name.text
    except:
        title = 'N/A'
    try:
        meta_score_element = driver.find_element(By.CSS_SELECTOR,'.xlarge > span:nth-child(3)')
        meta_score = meta_score_element.text
    except:
        meta_score = 'N/A'
    try:
        user_score_element = driver.find_element(By.CSS_SELECTOR,'div.large:nth-child(1)')
        user_score = user_score_element.text
    except:
        user_score = 'N/A'
    try:
        developer_element = driver.find_element(By.CSS_SELECTOR,'.button')
        developer = developer_element.text
    except:
        developer = 'N/A'
    try:
        platform_element = driver.find_element(By.CSS_SELECTOR,'.platform > a:nth-child(1)')
        platform = platform_element.text
    except:
        platform = 'N/A'
    try:
        toggle = driver.find_element(By.CSS_SELECTOR,'.product_summary > span:nth-child(2) > span:nth-child(1) > span:nth-child(4)')
        if toggle.text == "Expand":
            toggle.click()
    except:
        pass
    try:
        summary = driver.find_element(By.CSS_SELECTOR,'.product_summary > span:nth-child(2) > span:nth-child(1) > span:nth-child(2)')
        description = summary.text
    except:
        try:
            summary = driver.find_element(By.CSS_SELECTOR,'.product_summary > span:nth-child(2) > span:nth-child(1)')
            description = summary.text
        except:
            description = 'N/A'
    try:
        genre_list = driver.find_element(By.CSS_SELECTOR,'.product_genre')
        genre = genre_list.text
    except:
        genre = 'N/A'
    try:
        release_date_element = driver.find_element(By.CSS_SELECTOR,'.release_data > span:nth-child(2)')
        release_date = release_date_element.text
    except:
        release_date = 'N/A'
    try:
        rating_element = driver.find_element(By.CSS_SELECTOR,'li.summary_detail:nth-child(5) > span:nth-child(2)')
        rating = rating_element.text
    except:
        rating = 'N/A'
    return title, description, genre, meta_score, user_score, developer, platform, release_date, rating


In [6]:
# driver = webdriver.Firefox()
# Initialize the webdriver headlessly for faster scraping
# options = webdriver.FirefoxOptions()
# options.headless = True
# driver = webdriver.Firefox(options=options)
# driver.get('https://www.metacritic.com/game/xbox-one/ea-sports-ufc')
def web_driver():
    options = webdriver.FirefoxOptions()
#     options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    return driver

In [7]:
driver = web_driver()

driver.get('https://www.google.com')

In [8]:
# Check to see if any previous file has been saved to continue from there
start_index = 0
platform_data = []
try:
    # Try to read the previously saved csv file
    df = pd.read_csv(f"datasets/platform_description_merged/{save_file_name}.csv")
    # Get the index of the last row
    start_index = df.index[-1] + 1
    platform_data = df.to_dict('records')
except FileNotFoundError:
    # If the file doesn't exist, start from the first row
    start_index = 0

print('starting from index = ' + str(start_index))

starting from index = 401


In [9]:
# Start the scraping
start = time.time()
# iterate through all the rows
for index, row in tqdm(merged_df.iloc[start_index:].iterrows(), total=merged_df.shape[0] - start_index):
    url = row['url']
    title = row['title']
    # do your processing with the URL
    title, description, genre, meta_score, user_score, developer, platform, release_date, rating = get_data(driver,title,url)
    platform_data.append({
        'title': title,
        'description': description,
        'genres': genre,
        'meta_score' : meta_score,
        'user_score' : user_score,
        'developer' : developer,
        'platform' : platform,
        'release_date' : release_date,
        'rating': rating
    })

    # Save the data to the csv file every 100 iterations
    if index % 50 == 0:
        df = pd.DataFrame(data=platform_data, columns=platform_data[0].keys())
        df.to_csv(f"datasets/platform_description_merged/{save_file_name}.csv", index=False)
        
df = pd.DataFrame(data=platform_data, columns=platform_data[0].keys())
df.to_csv(f"datasets/platform_description_merged/{save_file_name}.csv", index=False)

end = time.time()
print(end - start)
driver.close()

  0%|                                                                             | 13/9240 [07:12<85:11:03, 33.24s/it]


WebDriverException: Message: Failed to decode response from marionette
